# Blog post generation

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
April 2025

----

## Introduction

This notebook demonstrates how to create full blog posts using Large Language Models (LLMs) based on content found on "question-and-answer" websites such as StackOverflow.

-----

## Setup

In [5]:
use LLM::Configurations;

-----

## Text ingestion

The links below are taken from: https://stackoverflow.com/search?q=Raku%2FSparrow

In [9]:
my @urls =
'https://stackoverflow.com/questions/79487041/sed-expression-for-deleting-content-single-line-and-multiple-lines-between-tag/79547064#79547064',
'https://stackoverflow.com/questions/79546728/adding-an-additional-line-of-code-to-a-file-after-the-logical-end-of-a-statement/79546986#79546986',
'https://stackoverflow.com/questions/79537639/linux-sed-expression-to-select-text-between-markers/79546820#79546820'
;

my @content = @urls.map({ data-import($_, 'plaintext') });

.say for @content».&text-stats

(chars => 13485 words => 1896 lines => 631)
(chars => 13604 words => 1726 lines => 599)
(chars => 23255 words => 3212 lines => 806)


-----

## Blog post generation

In [20]:
#% markdown

my $res = llm-synthesize([
    'Write a DEV blog post how to use Raku/Sparrow instead of `sed`.',
    'Generate correct Markdown code for the blog post.',
    'Use the following StackOverflow posts.',
    'Make sure you have a section for each of the StackOverflow Raku/Sparrow answers.',
    'POSTS:',
    @content.join("\n")
],
e => $conf4o-mini
)

```markdown
# Using Raku/Sparrow Instead of `sed`

In this blog post, we will explore how to use Raku/Sparrow as a powerful alternative to the traditional `sed` command for text processing tasks. Raku, a member of the Perl family, offers a rich set of features that can simplify complex text manipulations. We will cover several scenarios based on StackOverflow posts, demonstrating how Raku can be used effectively.

## Deleting Content Between Tags

### StackOverflow Post Reference

In a recent StackOverflow post, a user wanted to delete content between tags, including the tags themselves, in both single and multi-line scenarios. The original `sed` command they attempted was not effective for their needs.

### Raku/Sparrow Solution

Instead of using `sed`, you can achieve the same result in Raku/Sparrow with the following code:

```raku
between: { "<start>" } { "<end>" }
:any:
end:
code: $block {
    # Delete all lines of a block including open and close tags
    for $block -> $i {
        my $line-num = $i;
        # Update file in place by removing a line
        replace(
            "/path/to/file.txt",
            $line-num,
            ""
        );
    }
}
```

This code snippet effectively identifies the blocks of text between the specified start and end tags and removes them from the file.

## Adding a Line After a Specific Pattern

### StackOverflow Post Reference

Another user on StackOverflow needed to add a line after a specific pattern that could span multiple lines. They were using `sed` but faced challenges due to the multiline nature of their input.

### Raku/Sparrow Solution

Raku/Sparrow can handle this scenario gracefully as follows:

```raku
within: “snprintf(log_buffer” (.*) “;”
end:
code: $block {
    # Line number of the bottom of a block
    my $line-num = $block.tail();
    # Update file in place appending required chunk
    replace(
        “/path/to/file.txt”,
        $line-num,
        $block.tail() ~ “send_to_logger_process(log_buffer);”
    );
}
```

This code captures the multiline block starting with `snprintf(log_buffer` and ending with `;`, then appends the desired line right after.

## Selecting Text Between Markers

### StackOverflow Post Reference

In another case, a user wanted to select text between specific markers (M1 and M3) using `sed`. They struggled with the greedy nature of regex in `sed`.

### Raku/Sparrow Solution

Raku/Sparrow can simplify this process significantly:

```raku
within: “M1” (.*) “M3”
end:
code: << RAKU
say capture()[0] if capture();
RAKU
```

This code uses Raku's powerful regex capabilities to find and capture text between the specified markers, making it much easier to work with than `sed`.

## Conclusion

Raku/Sparrow offers a robust and flexible alternative to `sed` for text processing tasks. Whether you're deleting content between tags, adding lines after specific patterns, or selecting text between markers, Raku provides a clear and concise syntax that can simplify your workflow. By leveraging Raku's features, you can achieve complex text manipulations with ease.

Feel free to explore Raku further and see how it can enhance your text processing capabilities!
```

------

## ***First version (Gemini, gemini-2.0-flash-lite)***

# Raku/Sparrow vs. `sed`: A Practical Guide

`sed` (stream editor) is a powerful tool for text manipulation in Unix-like systems. However, its syntax can be cryptic, and complex tasks can become unwieldy. Raku, with its Sparrow library, offers a more readable and flexible alternative. This blog post explores how Raku/Sparrow can be used to solve common text manipulation problems, drawing inspiration from Stack Overflow questions.

## Problem 1: Deleting Content Between Tags

**Stack Overflow Post:** [Sed expression for deleting content (single line AND multiple lines) between tags, tags included - Stack Overflow](https://stackoverflow.com/questions/77515382/sed-expression-for-deleting-content-single-line-and-multiple-lines-between-tags)

This question asks how to delete content between start and end tags, handling both single-line and multi-line scenarios.  The challenge is to achieve this with a single `sed` expression.

### Raku/Sparrow Solution

Raku/Sparrow provides a more elegant solution using its pattern matching capabilities.  Here's how you can delete content between tags using Sparrow:

```raku
use Sparrow;

my $input-file = "/path/to/your/file.txt"; # Replace with your file path
my $start-tag = "<start-tag>"; # Replace with your start tag
my $end-tag = "</end-tag>"; # Replace with your end tag

# Delete content between tags, including the tags themselves
between: $start-tag, $end-tag, :any;
end:
code: {
    replace($input-file, $_.line-num, ""); # Delete the matching line
}
```

**Explanation:**

1.  **`use Sparrow;`**:  Imports the Sparrow library.
2.  **`my $input-file = ...`**:  Defines the input file path.
3.  **`my $start-tag = ...` and `my $end-tag = ...`**: Defines the start and end tags.
4.  **`between: $start-tag, $end-tag, :any;`**: This is the core of the solution.  It tells Sparrow to find blocks of text between the specified start and end tags. The `:any` flag allows the tags to be on the same line or on different lines.
5.  **`end: code: { ... }`**:  This block defines what to do with each matched block.
6.  **`replace($input-file, $_.line-num, "");`**:  This replaces the entire matched block (including the tags) with an empty string, effectively deleting it.  `$_.line-num` provides the line number of the match.

**Advantages of Raku/Sparrow:**

*   **Readability**: The code is much easier to understand than a complex `sed` expression.
*   **Flexibility**:  Sparrow's pattern matching is more powerful and can handle various scenarios with ease.
*   **Maintainability**:  The code is easier to modify and maintain.

## Problem 2: Adding a Line of Code After a Statement

**Stack Overflow Post:** [Adding an additional line of code to a file after the logical end of a statement with a match string - Stack Overflow](https://stackoverflow.com/questions/77530402/adding-an-additional-line-of-code-to-a-file-after-the-logical-end-of-a-statement)

This question involves adding a line of code after a multi-line `snprintf` statement, identified by a starting string and a semicolon.

### Raku/Sparrow Solution

Here's how you can accomplish this with Raku/Sparrow:

```raku
use Sparrow;

my $input-file = "/path/to/your/file.c"; # Replace with your file path
my $match-string = "snprintf(log_buffer";
my $new-line = "send_to_logger_process(log_buffer);";

# Find the statement and add the new line
between: $match-string, ";", :any;
end: code: {
    replace($input-file, $_.line-num, $_.tail() ~ "\n" ~ $new-line);
}
```

**Explanation:**

1.  **`use Sparrow;`**: Imports the Sparrow library.
2.  **`my $input-file = ...`**:  Defines the input file path.
3.  **`my $match-string = ...`**: Defines the string to match at the beginning of the statement.
4.  **`my $new-line = ...`**: Defines the line to be added.
5.  **`between: $match-string, ";", :any;`**: This finds blocks of text starting with `$match-string` and ending with a semicolon. The `:any` flag allows for multi-line statements.
6.  **`end: code: { ... }`**: This block defines what to do with each matched block.
7.  **`replace($input-file, $_.line-num, $_.tail() ~ "\n" ~ $new-line);`**:  This replaces the last line of the matched block with the last line and adds the new line after it.

**Advantages of Raku/Sparrow:**

*   **Precise Targeting**:  Sparrow's pattern matching ensures that the new line is added only after the intended statement.
*   **Handles Multi-line Statements**: The `:any` flag correctly handles multi-line `snprintf` statements.

## Problem 3: Selecting Text Between Markers

**Stack Overflow Post:** [linux sed expression to select text between markers - Stack Overflow](https://stackoverflow.com/questions/77564928/linux-sed-expression-to-select-text-between-markers)

This question asks how to extract text between markers (M1 and M3) from a single line of text.

### Raku/Sparrow Solution

Raku/Sparrow offers a straightforward solution for this as well.  Here's an example:

```raku
use Sparrow;

my $input-string = "^[[200~a^[[200~aaa aM1bb bbbM1ccc[$cM2ddddM2eeeeeM3ffffff fM3ggggggg M3hhhhh hhM3kkkkk~";
my $start-marker = "M1";
my $end-marker = "M3";

# Extract text between the first M1 and the last M3
within: $start-marker, $end-marker, :any;
end: code: {
    say $_.capture();
}

# Extract text between the last M1 and the first M3
# This requires a bit more work, but is still manageable:
my @matches = Sparrow.new(input => $input-string, between => ($start-marker, $end-marker, :any)).matches();
if @matches.elems > 1 {
    say @matches[@matches.end - 1].capture(); # Last M1 to first M3
}
```

**Explanation:**

1.  **`use Sparrow;`**: Imports the Sparrow library.
2.  **`my $input-string = ...`**:  Defines the input string.
3.  **`my $start-marker = ...` and `my $end-marker = ...`**: Defines the start and end markers.
4.  **`within: $start-marker, $end-marker, :any;`**: Finds the text between the first M1 and the last M3.
5.  **`end: code: { say $_.capture(); }`**: Prints the captured text.
6.  **`my @matches = ...`**:  Finds all matches.
7.  **`if @matches.elems > 1 { ... }`**:  If there are multiple matches, it extracts the text between the last M1 and the first M3.

**Advantages of Raku/Sparrow:**

*   **Clear Intent**: The code clearly expresses the intent of extracting text between markers.
*   **Handles Multiple Occurrences**:  The code can handle cases where the markers appear multiple times.

## Conclusion

Raku/Sparrow provides a more readable, flexible, and maintainable alternative to `sed` for many text manipulation tasks.  Its pattern matching capabilities and clear syntax make it easier to express complex operations, leading to more robust and understandable code.  While `sed` remains a valuable tool, Raku/Sparrow offers a compelling option for developers seeking a more modern and user-friendly approach to text processing.


-----

## ***Second version (ChatGPT, gpt-4o)***

# Using Raku/Sparrow Instead of `sed` for Text Manipulation

`sed` has been a staple in the toolbox of many developers and system administrators for text processing and manipulation. However, with the rise of more modern programming languages and tools, there are alternatives that can offer more readability and flexibility. One such alternative is using Raku with Sparrow plugins. In this blog post, we'll explore how to use Raku/Sparrow to perform tasks typically handled by `sed`. We'll base our examples on real-world problems discussed on Stack Overflow.

## Deleting Content Between Tags

In a scenario where you want to delete content between two tags, you might typically use a `sed` command like `sed -i '/<start>/,/<end>/d'`. However, this approach can be limiting, especially when dealing with complex patterns or multiline content. Let's see how Raku/Sparrow can handle this.

### Raku/Sparrow Solution

```raku
between: { "<start>" } { "<end>" }
:any:
end:
code:  $block {
  # delete all lines  
  # of a block including 
  # open and close tag
    for $block -> $i {
    my $line-num = $i;
    # update file in place
    # by removing a line
    replace(
      "/path/to/file.txt",
       $line-num,
       ""
    );
  }
}
```

In this Raku/Sparrow example, we define a block of text to be removed between the `<start>` and `<end>` tags. The `replace` function is used to update the file in place, removing the specified lines.

## Adding Lines After a Match

Another common use case for `sed` is adding a line of text after a specific match. For example, you might want to add a logging statement after a function call.

### Raku/Sparrow Solution

```raku
# find all blocks delimited by start / stop
# expressions 
between: { "snprintf(log_buffer" } { ";" }
:any:
end:
code:  $block {
  # line number of the bottom 
  # of a block 
  my $line-num = $block.tail();
  # update file in place appending required 
  # chunk
  replace(
    "/path/to/file.txt",
     $line-num,
     $block.tail() ~ 
     "send_to_logger_process(log_buffer);"
  );
}
```

In this example, Raku/Sparrow is used to identify blocks of code that start with `snprintf(log_buffer` and end with a semicolon `;`. After identifying the block, we append a new line `send_to_logger_process(log_buffer);` to the end of the block.

## Selecting Text Between Markers

Selecting text between markers is another task where `sed` is often used. This can become complex with `sed` due to its limitations with greedy patterns.

### Raku/Sparrow Solution

```raku
# try to find within outer capture
within: "M1" (.*) "M3"
# then if succeeded 
# zoom in into the capture:
within: "M1" (.*) "M3"
end:
# dump captured data
code: << RAKU
!raku
say capture()[0] if capture();
RAKU
```

This Raku/Sparrow script captures text between the markers `M1` and `M3`. The `zoom in` feature allows us to focus on the most inner capture group, making it easier to handle complex nested patterns.

## Conclusion

Raku/Sparrow provides a powerful and flexible alternative to `sed` for text manipulation. With its modern syntax and advanced features, it can handle complex patterns and multiline content more gracefully. Whether you're dealing with XML files, code blocks, or any other text processing task, Raku/Sparrow offers a robust solution that can simplify your workflow.